In [13]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [14]:
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures
import tensorflow as tf
import pandas as pd
import re

2023-02-01 04:23:55.951883: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-01 04:23:55.952980: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-01 04:23:55.953639: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-01 04:23:55.956365: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [15]:
df = pd.read_csv("/kaggle/input/my-data/my_sample.txt")#/kaggle/input/my-data
df.columns = ["called_function","argument_1","argument_2","type_of_argument","formal_parameter_1","formal_parameter_2","label"]

In [16]:
df.isna().sum()

called_function         10
argument_1              88
argument_2              88
type_of_argument         0
formal_parameter_1    7579
formal_parameter_2    7579
label                    0
dtype: int64

In [17]:
df['called_function'] = df['called_function'].fillna('unknown')
df['argument_1']=df['argument_1'].fillna('unknown')
df['argument_2']=df['argument_2'].fillna('unknown')
df['type_of_argument']=df['type_of_argument'].fillna('unknown')
df['formal_parameter_1']=df['formal_parameter_1'].fillna('unknown')
df['formal_parameter_2']=df['formal_parameter_2'].fillna('unknown')

In [18]:
df.isna().sum()

called_function       0
argument_1            0
argument_2            0
type_of_argument      0
formal_parameter_1    0
formal_parameter_2    0
label                 0
dtype: int64

In [19]:
df['total'] = df['called_function'] + '<SEP>'+ df['argument_1'] + '<SEP>'+df['type_of_argument'] + '<SEP>'+ df['formal_parameter_1'] + '<SEP>'+df['formal_parameter_2']

In [20]:
df.drop(['called_function','argument_1','argument_2','type_of_argument','formal_parameter_1','formal_parameter_2'],axis=1,inplace=True)

In [21]:
df = df.sample(frac = 1) # shuffling the dataset
df.reset_index(drop=True,inplace=True)

In [22]:
df

,label,total
0,0,expect_string<SEP>unknown<SEP>int<SEP>unknown<...
1,0,setenv<SEP>ethaddr<SEP>int<SEP>unknown<SEP>unk...
2,0,sysfs_attr_get_value<SEP>devpath<SEP>int<SEP>u...
3,0,CYGACC_COMM_IF_PUTC<SEP>*<SEP>void<SEP>x<SEP>y
4,1,CYGACC_COMM_IF_PUTC<SEP>0x18<SEP>void<SEP>x<SEP>y
...,...,...
10638,1,strcmp<SEP>callback<SEP>int<SEP>_Str1<SEP>_Str2
10639,0,occi_create_element<SEP>occi.vm.provider<SEP>i...
10640,0,strcmp<SEP>key<SEP>int<SEP>_Str1<SEP>_Str2
10641,1,setenv<SEP>fatload ide 0 8000000 uimage.karef;...


In [23]:
classifier = TFBertForSequenceClassification.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [25]:
classifier.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 109,483,778
Trainable params: 109,483,778
Non-trainable params: 0
_________________________________________________________________


In [26]:
m = len(df)
sample = int(m/10)

In [27]:
train_df = df[sample:]
test_df = df[:sample]

In [28]:
def convert_data_to_examples(train, test, code_snippet, label_of_code_snippet): 
  train_InputExamples = train.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[code_snippet], 
                                                          text_b = None,
                                                          label = x[label_of_code_snippet]), axis = 1)

  validation_InputExamples = test.apply(lambda x: InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this case
                                                          text_a = x[code_snippet], 
                                                          text_b = None,
                                                          label = x[label_of_code_snippet]), axis = 1)
  
  return train_InputExamples, validation_InputExamples

In [29]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train_df, 
                                                                           test_df, 
                                                                           'total', 
                                                                           'label')

In [30]:
train_df

,label,total
1064,0,generateModuleFile<SEP>moduleName<SEP>int<SEP>...
1065,1,_difftime64<SEP>_Time2<SEP>double<SEP>_Time1<S...
1066,0,uaw_lstrcmpW<SEP>String1<SEP>int<SEP>String1<S...
1067,1,_hypotf<SEP>_Y<SEP>float<SEP>_X<SEP>_Y
1068,0,ACPI_OPTION<SEP>-n <Signature><SEP>int<SEP>unk...
...,...,...
10638,1,strcmp<SEP>callback<SEP>int<SEP>_Str1<SEP>_Str2
10639,0,occi_create_element<SEP>occi.vm.provider<SEP>i...
10640,0,strcmp<SEP>key<SEP>int<SEP>_Str1<SEP>_Str2
10641,1,setenv<SEP>fatload ide 0 8000000 uimage.karef;...


In [31]:
test_df

,label,total
0,0,expect_string<SEP>unknown<SEP>int<SEP>unknown<...
1,0,setenv<SEP>ethaddr<SEP>int<SEP>unknown<SEP>unk...
2,0,sysfs_attr_get_value<SEP>devpath<SEP>int<SEP>u...
3,0,CYGACC_COMM_IF_PUTC<SEP>*<SEP>void<SEP>x<SEP>y
4,1,CYGACC_COMM_IF_PUTC<SEP>0x18<SEP>void<SEP>x<SEP>y
...,...,...
1059,1,strcmp<SEP>php<SEP>int<SEP>unknown<SEP>unknown
1060,0,setenv<SEP>keybd<SEP>int<SEP>unknown<SEP>unknown
1061,0,uaw_lstrcmpW<SEP>String1<SEP>int<SEP>String1<S...
1062,0,strcmp<SEP>rolestatus<SEP>int<SEP>unknown<SEP>...


In [32]:
def convert_examples_to_tf_dataset(examples, tokenizer, max_length=128):
    features = [] # -> will hold InputFeatures to be converted later

    for e in examples:
        # Documentation is really strong for this method, so please take a look at it
        input_dict = tokenizer.encode_plus(
            e.text_a,
            add_special_tokens=True,
            max_length=max_length, # truncates if len(s) > max_length
            return_token_type_ids=True,
            return_attention_mask=True,
            pad_to_max_length=True, # pads to the right by default # CHECK THIS for pad_to_max_length
            truncation=True
        )

        input_ids, token_type_ids, attention_mask = (input_dict["input_ids"],
            input_dict["token_type_ids"], input_dict['attention_mask'])

        features.append(
            InputFeatures(
                input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, label=e.label
            )
        )

    def gen():
        for f in features:
            yield (
                {
                    "input_ids": f.input_ids,
                    "attention_mask": f.attention_mask,
                    "token_type_ids": f.token_type_ids,
                },
                f.label,
            )

    return tf.data.Dataset.from_generator(
        gen,
        ({"input_ids": tf.int32, "attention_mask": tf.int32, "token_type_ids": tf.int32}, tf.int64),
        (
            {
                "input_ids": tf.TensorShape([None]),
                "attention_mask": tf.TensorShape([None]),
                "token_type_ids": tf.TensorShape([None]),
            },
            tf.TensorShape([]),
        ),
    )

In [33]:
code_snippet = 'total'
label_of_code_snippet = 'label'

In [34]:
train_InputExamples, validation_InputExamples = convert_data_to_examples(train_df, test_df, code_snippet, label_of_code_snippet)

train_data = convert_examples_to_tf_dataset(list(train_InputExamples), tokenizer)
train_data = train_data.shuffle(100).batch(32).repeat(2)

validation_data = convert_examples_to_tf_dataset(list(validation_InputExamples), tokenizer)
validation_data = validation_data.batch(32)

/opt/conda/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2307: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [35]:
# classifier.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0), 
#               loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
#               metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')])

In [36]:
# classifier.fit(train_data, epochs=30, validation_data=validation_data,verbose=1,batch_size=64,steps_per_epoch=166)

Epoch 1/30


2023-02-01 04:25:11.144621: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


166/166 [==============================] - 116s 535ms/step - loss: 0.6917 - accuracy: 0.5169 - val_loss: 0.6511 - val_accuracy: 0.6118
Epoch 2/30
166/166 [==============================] - 86s 517ms/step - loss: 0.4926 - accuracy: 0.7539 - val_loss: 0.4146 - val_accuracy: 0.8224
Epoch 3/30
166/166 [==============================] - 86s 519ms/step - loss: 0.3311 - accuracy: 0.8605 - val_loss: 0.2989 - val_accuracy: 0.8731
Epoch 4/30
166/166 [==============================] - 55s 329ms/step - loss: 0.2549 - accuracy: 0.8936 - val_loss: 0.2840 - val_accuracy: 0.8900


In [10]:
param_grid = {
    "per_gpu_batch_size": [16, 32,64],
    "learning_rate": [5e-6, 1e-5, 5e-5],
    "num_epochs": [5, 10, 25,40]
    "steps_per_epoch":[2128,1064,425,266]
}

SyntaxError: invalid syntax (3460627099.py, line 5)

In [11]:
from sklearn import model_selection

In [12]:
model = model_selection.GridSearchCV(
    estimator=classifier,
    param_grid=param_grid,
    scoring="accuracy",
    verbose=1,
    n_jobs=2,
    cv=5
)

NameError: name 'classifier' is not defined

In [ ]:
model.fit(train_data)